In [78]:
import re
book='matt'
booklong='matthew'
inF=open("txts/"+book+".txt","r")
test=inF.read()

In [79]:
t=re.sub('(\([^\)]*)(\d+)([\D]*\))','',test)
chs=t.split('|')
len(chs)

28

In [80]:
#len(chs[0].trim())
import json, time ,re

In [81]:
#create index for chapters on elasticsearch

import elasticsearch
index_name = "bv"
index_type = "bible_verses"
elastic_host = "gaia.hud.ac.uk"
elastic_port = "9200"
# Connect to ElasticSearch
es = elasticsearch.Elasticsearch(elastic_host, port=elastic_port)
# Delete & Create index

In [82]:
def getimgs(bk,ch,v):
    try:
        return es.search(index="bchap_map", body={"from":0,"query":{"bool":{"filter":[{"bool":{"must":[{"bool":{"must":[{"match_phrase":{"book":{"query":bk,"slop":0,"boost":1.0}}},{"bool":{"should":[{"bool":{"must":[{"match_phrase":{"sc":{"query":ch,"slop":0,"boost":1.0}}},{"range":{"ssv":{"from":None,"to":v,"include_lower":True,"include_upper":True,"boost":1.0}}},{"range":{"sev":{"from":v,"to":None,"include_lower":True,"include_upper":True,"boost":1.0}}}],"adjust_pure_negative":True,"boost":1.0}},{"bool":{"must":[{"match_phrase":{"ec":{"query":ch,"slop":0,"boost":1.0}}},{"range":{"esv":{"from":None,"to":v,"include_lower":True,"include_upper":True,"boost":1.0}}},{"range":{"eev":{"from":v,"to":None,"include_lower":True,"include_upper":True,"boost":1.0}}}],"adjust_pure_negative":True,"boost":1.0}}],"adjust_pure_negative":True,"boost":1.0}}],"adjust_pure_negative":True,"boost":1.0}}],"adjust_pure_negative":True,"boost":1.0}}],"adjust_pure_negative":True,"boost":1.0}},"sort":[{"sno":{"order":"asc"}}]})
    except:
        print('adding..',bk,ch,':',v)


In [83]:
verses=re.findall('\D+',chs[2])
vnos=re.findall('\d+',chs[2])
chno=re.match('^\d+',chs[2]).group() 
#returns matched sstring re.start() is start re.end() end index, re.span() start and end tuple
verx=chs[2][re.match('^\d+',chs[2]).end():]


In [84]:
#chno

In [102]:
#vnos
getimgs('matt',2,24)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [], 'max_score': None, 'total': 0},
 'timed_out': False,
 'took': 0}

In [86]:
#verses

In [103]:
def uv(cn,vn):
    cno=str(chno).zfill(2)
    vno=str(vnos[vn]).zfill(2)
    #verx=chs[i][re.match('^\d+',chs[i]).end():]
    #import numpy as np
    #x = np.array(vnos)
    #vnos = x.astype(np.int).tolist()
    ims=getimgs(bk=book,ch=int(chno),v=int(vnos[vn]))
    if ims['hits']['total']==0:
        print(book,chno,':',vnos[vn], 'returned null')
        return
    im=ims['hits']['hits'][0]['_source']
    js={
        "bookcode":book, 
        "book":booklong,
        "chno":int(chno),
        "vno":int(vnos[vn]),
        "verse":verses[vn].strip(),
        "img1":im['img1'],
        "img2":im['img2']
    }
    data=json.dumps(js)
    id=book+'_'+cno+'_'+vno
    #POST TO ElasticSearch
    res = es.index(index=index_name, doc_type=index_type, id=id, body=data)
    #print(verses[vn])
    if res['_shards']['successful']==0:
        print(id, "..created!" if res['_shards']['successful']>0 else "failed!")
    #print(id, "..created!" if res['_shards']['successful']>0 else "failed!")

In [104]:
es.indices.delete(index=index_name, ignore=[400, 404])
es.indices.create(index=index_name, ignore=400)
for i in range(len(chs)):
    if i==0:
        continue
    chno=int(re.match('^\d+',chs[i]).group())
    verses=re.findall('\D+',chs[i])
    vnos=re.findall('\d+',chs[i])
    vnos[0]="1"
    print("processing chapt ",chno,"...")
    if len(verses)!=len(vnos):
        print( "verse no mismatch for",chno)
        continue
    if i>0:
        for j in range(len(vnos)):
            uv(chno,j)
print("done!")

processing chapt  1 ...
processing chapt  2 ...
processing chapt  3 ...
processing chapt  4 ...
processing chapt  5 ...
processing chapt  6 ...
processing chapt  7 ...
processing chapt  8 ...
matt 8 : 24 returned null
processing chapt  9 ...
processing chapt  10 ...
processing chapt  11 ...
processing chapt  12 ...
processing chapt  13 ...
processing chapt  14 ...
matt 14 : 15 returned null
matt 14 : 13 returned null
matt 14 : 14 returned null
matt 14 : 15 returned null
matt 14 : 17 returned null
matt 14 : 18 returned null
matt 14 : 19 returned null
matt 14 : 20 returned null
matt 14 : 21 returned null
matt 14 : 37 returned null
matt 14 : 38 returned null
matt 14 : 39 returned null
processing chapt  16 ...
processing chapt  17 ...
processing chapt  18 ...
processing chapt  19 ...
processing chapt  20 ...
processing chapt  21 ...
matt 21 : 33 returned null
matt 21 : 34 returned null
matt 21 : 35 returned null
matt 21 : 36 returned null
matt 21 : 37 returned null
matt 21 : 39 returned nu

In [ ]:
es.get(index='bv', doc_type='bible_verses', id='matt_28_19')['_source']